# Assignment Submission - Sector Analysis For Portfolio Construction
***
#### This implementation is only for predicting the price of NIFTY AUTO INDEX for year 2020 and 2021 based on the prices and indicator values from 2011 - 2019

#### First we fetch the yearly closing data of NIFTY AUTO INDEX for years 2011 - 2021

#### (Summary and thoughts at the end)

In [143]:
import yfinance as yf
import pandas as pd

# Define the ticker symbol for Nifty Auto Index
ticker_symbol = '^CNXAUTO'

# Set the start and end dates for the data
start_date = '2011-01-01'
end_date = '2022-01-01'

# Fetch the data
nifty_auto_data = yf.download(ticker_symbol, start=start_date, end=end_date)

# Keep only the 'Close' column and reset the index
nifty_auto_prices = nifty_auto_data['Close'].reset_index()

# Extract year from the 'Date' column
nifty_auto_prices['Year'] = pd.to_datetime(nifty_auto_prices['Date']).dt.year

# Group by year and calculate the mean close price for each year
nifty_auto_prices_grouped = nifty_auto_prices.groupby('Year')['Close'].mean().reset_index()

# Rename columns for consistency
nifty_auto_prices_grouped.columns = ['Year', 'Price']

# Display the final dataframe
nifty_auto_prices_grouped


[*********************100%%**********************]  1 of 1 completed


,Year,Price
0,2011,3606.889831
1,2012,4112.445713
2,2013,4700.616335
3,2014,6771.148966
4,2015,8358.188844
5,2016,8771.269498
6,2017,10597.403446
7,2018,10597.266697
8,2019,8011.832023
9,2020,7339.960393


#### Then we fetch data on various indicators like Inflation, GDP, Oil per GDP, interest rate and other relavant indicators from World Bank Data

In [144]:
from wbdata import get_data
from datetime import datetime
import pandas as pd

# Define the country code for India
country = 'IND'

# Define the start and end years
start_year = 2011
end_year = 2021

# Convert the years to datetime objects
start_date = datetime(start_year, 1, 1)  # January 1st of the start year
end_date = datetime(end_year, 1, 1)  # January 1st of the end year

# Define the indicator codes for each macroeconomic variable
indicators = {
    'Exchange Rate': 'PA.NUS.FCRF',
    'Inflation': 'FP.CPI.TOTL.ZG',
    'GDP': 'NY.GDP.MKTP.CD',
    'Net Trade': 'NE.TRD.GNFS.ZS',  # Trade (% of GDP)
    'Oil per GDP': 'NY.GDP.PETR.RT.ZS',  # Oil rents (% of GDP)
    'Real Interest Rate': 'FR.INR.RINR',
    'S&P 500 Index': 'SP.DYN.TFRT.IN',
    'Unemployment': 'SL.UEM.TOTL.ZS',
    'Interest Rates': 'FR.INR.RINR',
    # 'Electric Vehicle Adoption': 'EN.ATM.CO2E.KT'
}

# Fetch the data for each indicator
data = {}
for indicator_name, indicator_code in indicators.items():
    try:
        indicator_data = get_data(indicator_code, country, date=(start_date, end_date))
        simplified_data = [{'Year': int(item['date']), 'value': item['value']} for item in indicator_data]
        data[indicator_name] = simplified_data
    except Exception as e:
        print(f"Error fetching data for {indicator_name}: {e}")

# Convert the data to DataFrame format
dfs = []
for indicator_name, indicator_data in data.items():
    df = pd.DataFrame(indicator_data)
    df.rename(columns={'value': indicator_name}, inplace=True)
    dfs.append(df)

# Merge the DataFrames on the 'year' column
merged_df = dfs[0]
for df in dfs[1:]:
    merged_df = pd.merge(merged_df, df, on='Year', how='outer')

# Display the combined DataFrame
merged_df


,Year,Exchange Rate,Inflation,GDP,Net Trade,Oil per GDP,Real Interest Rate,S&P 500 Index,Unemployment,Interest Rates
0,2021,73.918013,5.131407,3.150307e+12,45.667683,0.325610,0.147236,2.031,6.380,0.147236
1,2020,74.099567,6.623437,2.671595e+12,37.804125,0.144344,4.203910,2.051,7.860,4.203910
2,2019,70.420341,3.729506,2.835606e+12,39.905404,0.305028,6.894875,2.109,6.510,6.894875
3,2018,68.389467,3.938826,2.702930e+12,43.616969,0.446630,5.361666,2.177,7.650,5.361666
4,2017,65.121569,3.328173,2.651474e+12,40.742497,0.313225,5.327609,2.200,7.728,5.327609
5,2016,67.195313,4.948216,2.294797e+12,40.082486,0.220298,6.232711,2.273,7.808,6.232711
6,2015,64.151944,4.906973,2.103588e+12,41.922914,0.359941,7.556488,2.287,7.891,7.556488
7,2014,61.029514,6.665657,2.039126e+12,48.922186,0.945193,6.695176,2.306,7.976,6.695176
8,2013,58.597845,10.017878,1.856722e+12,53.844132,1.193354,3.865993,2.406,8.060,3.865993
9,2012,53.437233,9.478997,1.827638e+12,55.793722,1.313872,2.473522,2.467,8.138,2.473522


#### We also incorporate the data from Indian Fama French and Momentum Factors - Yearly

In [145]:
import pandas as pd

# Load FFMR.csv into a DataFrame
ffmr_data = pd.read_csv('4m/FFMR.csv')

# Filter data for years 2010 to 2021
ffmr_filtered = ffmr_data[(ffmr_data['Date'] >= 2010) & (ffmr_data['Date'] <= 2021)]

# Merge with your existing input data (assuming 'merged_df' is your input data DataFrame)
merged_df = pd.merge(merged_df, ffmr_filtered, left_on='Year', right_on='Date', how='inner')

# Drop the redundant 'Date' column after merging
merged_df.drop('Date', axis=1, inplace=True)

# Now 'merged_input_data' contains the merged data from 2010 to 2021 for your regression task
merged_df


,Year,Exchange Rate,Inflation,GDP,Net Trade,Oil per GDP,Real Interest Rate,S&P 500 Index,Unemployment,Interest Rates,SMB,HML,WML,MF,RF
0,2021,73.918013,5.131407,3.150307e+12,45.667683,0.325610,0.147236,2.031,6.380,0.147236,13.736321,40.396508,33.472831,38.794695,3.378763
1,2020,74.099567,6.623437,2.671595e+12,37.804125,0.144344,4.203910,2.051,7.860,4.203910,3.499843,9.452417,-4.071184,23.997248,3.762687
2,2019,70.420341,3.729506,2.835606e+12,39.905404,0.305028,6.894875,2.109,6.510,6.894875,-24.617654,-21.862984,31.302765,-1.890484,5.832885
3,2018,68.389467,3.938826,2.702930e+12,43.616969,0.446630,5.361666,2.177,7.650,5.361666,-22.114119,-20.405352,22.361076,-13.312374,6.588476
4,2017,65.121569,3.328173,2.651474e+12,40.742497,0.313225,5.327609,2.200,7.728,5.327609,10.831492,28.019028,16.890506,30.638893,6.120388
5,2016,67.195313,4.948216,2.294797e+12,40.082486,0.220298,6.232711,2.273,7.808,6.232711,-8.975860,8.737235,-1.532868,-1.370627,6.725270
6,2015,64.151944,4.906973,2.103588e+12,41.922914,0.359941,7.556488,2.287,7.891,7.556488,18.322080,14.538864,25.438795,-2.640981,7.677076
7,2014,61.029514,6.665657,2.039126e+12,48.922186,0.945193,6.695176,2.306,7.976,6.695176,45.107351,-2.420965,11.170882,24.250721,8.660956
8,2013,58.597845,10.017878,1.856722e+12,53.844132,1.193354,3.865993,2.406,8.060,3.865993,-23.549635,3.245670,26.204773,-2.124655,8.623270
9,2012,53.437233,9.478997,1.827638e+12,55.793722,1.313872,2.473522,2.467,8.138,2.473522,6.211794,1.857217,-6.152748,14.640541,8.451609


#### Then we finally append the Years prices as well with our training data as target variable

In [146]:
# Assuming merged_df and nifty_auto_prices_grouped are already defined

# Merge with merged_df to include the 'Price' column
merged_df = pd.merge(merged_df, nifty_auto_prices_grouped, on='Year', how='left')
merged_df



,Year,Exchange Rate,Inflation,GDP,Net Trade,Oil per GDP,Real Interest Rate,S&P 500 Index,Unemployment,Interest Rates,SMB,HML,WML,MF,RF,Price
0,2021,73.918013,5.131407,3.150307e+12,45.667683,0.325610,0.147236,2.031,6.380,0.147236,13.736321,40.396508,33.472831,38.794695,3.378763,10486.558430
1,2020,74.099567,6.623437,2.671595e+12,37.804125,0.144344,4.203910,2.051,7.860,4.203910,3.499843,9.452417,-4.071184,23.997248,3.762687,7339.960393
2,2019,70.420341,3.729506,2.835606e+12,39.905404,0.305028,6.894875,2.109,6.510,6.894875,-24.617654,-21.862984,31.302765,-1.890484,5.832885,8011.832023
3,2018,68.389467,3.938826,2.702930e+12,43.616969,0.446630,5.361666,2.177,7.650,5.361666,-22.114119,-20.405352,22.361076,-13.312374,6.588476,10597.266697
4,2017,65.121569,3.328173,2.651474e+12,40.742497,0.313225,5.327609,2.200,7.728,5.327609,10.831492,28.019028,16.890506,30.638893,6.120388,10597.403446
5,2016,67.195313,4.948216,2.294797e+12,40.082486,0.220298,6.232711,2.273,7.808,6.232711,-8.975860,8.737235,-1.532868,-1.370627,6.725270,8771.269498
6,2015,64.151944,4.906973,2.103588e+12,41.922914,0.359941,7.556488,2.287,7.891,7.556488,18.322080,14.538864,25.438795,-2.640981,7.677076,8358.188844
7,2014,61.029514,6.665657,2.039126e+12,48.922186,0.945193,6.695176,2.306,7.976,6.695176,45.107351,-2.420965,11.170882,24.250721,8.660956,6771.148966
8,2013,58.597845,10.017878,1.856722e+12,53.844132,1.193354,3.865993,2.406,8.060,3.865993,-23.549635,3.245670,26.204773,-2.124655,8.623270,4700.616335
9,2012,53.437233,9.478997,1.827638e+12,55.793722,1.313872,2.473522,2.467,8.138,2.473522,6.211794,1.857217,-6.152748,14.640541,8.451609,4112.445713


#### Now we import the necessary libraries

In [147]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


#### This step involves splitting the data so that the last 2 years are treated as the test data and the rest of the years are treated as training data. 

In [138]:
# Splitting data into training and testing sets
train_data = merged_df[merged_df['Year'].between(2011, 2019)]
test_data = merged_df[merged_df['Year'].between(2020, 2021)]

# Extracting the prices for training and testing
train_prices = train_data['Price'].values.reshape(-1, 1)
test_prices = test_data['Price'].values.reshape(-1, 1)

# Scaling the data using MinMaxScaler
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_prices)
test_scaled = scaler.transform(test_prices)


#### Then we setup the LSTM model with lookback value to 5 and prepare data as well

In [148]:
# Define the number of time steps (years) to look back for the LSTM model
n_steps = 5  # You can adjust this based on the time series patterns

# Function to prepare the data for LSTM
def prepare_data(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data[i:(i + n_steps), 0])
        y.append(data[i + n_steps, 0])
    return np.array(X), np.array(y)

# Prepare the training data for LSTM
X_train, y_train = prepare_data(train_scaled, n_steps)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))  # Reshape for LSTM input


#### LSTM is a very strong recurring neural network implementation and thus it can recognise patterns in a time series easily. I setup a single layer because increasing layers lead to overfitting of the data. Then we start the fitting process

In [140]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Train the LSTM model
model.fit(X_train, y_train, epochs=100, verbose=0)  # Adjust epochs as needed


#### No we test our given model

In [141]:
# Prepare the testing data for prediction
inputs = np.vstack((train_scaled[-n_steps:], test_scaled))
X_test, y_test = prepare_data(inputs, n_steps)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))  # Reshape for LSTM input

# Predict prices for 2022 and 2023
predicted_scaled = model.predict(X_test)
predicted_prices = scaler.inverse_transform(predicted_scaled.reshape(-1, 1))

# Display the predicted prices for 2022 and 2023
predicted_df = pd.DataFrame({'Year': test_data['Year'].values[-2:], 'Predicted Price': predicted_prices.flatten()})
print(predicted_df)


1/1 [==============================] - 0s 245ms/step
   Year  Predicted Price
0  2021      8746.704102
1  2020      8133.392578


### Why use LSTM and not ARIMA
***
I chose to use LSTM over ARIMA because 
1) It was preventing me from using the indicator values at all, so had to include indicators in exogenous variables parameter
2) I found that the grid search of p,q,d parameters could not get me a good parameter. 
3) Even if I eyeballed p,q,d parameters, it performed poorly.

I would have to research upon ARIMA more in order to get used to its implementation

LSTM on the other hand, I am quite used to it. It gave me more accurate results. Also by incorporating just a single layer and also dropout regularisation, we can further decrease overfitting as well.

Since we did not have access to a larger dataset (we just had yearly data and not, say monthly, for the indicators), we could not get a more accurate generalisation, but I found it to be satisfactory